<a href="https://colab.research.google.com/github/karsarobert/Deep-Learning-2022/blob/main/10/PTE_DL10_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Szöveggenerálás RNN-nel

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/text/tutorials/text_generation"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/text/blob/master/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/text/blob/master/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/text/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

Ez a bemutató azt mutatja be, hogyan lehet szöveget generálni egy karakteralapú RNN segítségével. Rejtő Jenő írásaiból álló adatkészlettel fog dolgozni, amely ötlet Andrej Karpathy The Unreasonable Effectiveness of Recurrent Neural Networks című könyvéből származik. Adott egy karaktersorozat ebből az adatból, képezzünk ki egy modellt a sorozat következő karakterének ("e") előrejelzésére. A modell ismételt meghívásával hosszabb szövegsorozatok generálhatók.

Megjegyzés: Engedélyezze a GPU-gyorsítást a notebook gyorsabb végrehajtásához. A Colab: Futtatási idő > Futtatási idő típusának módosítása > Hardveres gyorsító > GPU.

Ez a bemutató tf.keras és eager execution használatával megvalósított futtatható kódot tartalmaz. A következő a mintakimenet, amikor az ebben a bemutatóban szereplő modell 30 epochán keresztül képzett, és a "A" felszólítással indult:

<pre>
An saját magaszására társal lett van a pitánsággal, könnyedéseket.
A herceg szeme le régen is tűt. Mehes. Fülig Jimmy? Mi ajra, ha Hurcunk Fernántesz! És igen ötök boripán az találgassza, Felség, angol hibetkézik az én nevemben, azt hitte, hogy néhány óráj baj vas, hát de nem hites, előfordul egy erőtel sem mozdult. Pedig a hajón vérn A fiú uralkodásai és lenéssel az illetőkeá.
- Hány és GrAndszervez erted?
Így öreg Wilson Hutchins (az amerikai fűtő már csak azért sem vonállattag érezte, amelyen a pincér aztán ki történt. Fel tudum. Most már közzem (Övig nem ívás, hanem Jiment furcsa, akkor ja mokdanás, mert a brót és fél személyesen ismerem.
- Warins a vőlegverék el - mondja a stamát... Az is a pillanatra! - jegyezte meg közölte?
- Ezent lesz a borízsal állt. - Es osztálybal. Öngyen közt kenyeres tudja, ő a próféte megtoválját, így szólt:
- Integessen Felség, hogy egy nap alatt sok mindent tett, embere! Hozzt, hogy nem keresem tovább vezeteti Bannera. Tevissza kelé krabitány az á 

</pre>

Bár néhány mondat nyelvtanilag helyes, a legtöbbnek nincs értelme. A modell nem tanulta meg a szavak jelentését, de fontolja meg:

* A modell karakteralapú. Amikor a képzés elkezdődött, a modell nem tudta, hogyan kell egy szót leírni, vagy hogy a szavak egyáltalán a szöveg egységei.

* Amint azt az alábbiakban bemutatjuk, a modell kis szövegrészleteken (egyenként 100 karakter) tanul, és még mindig képes egy hosszabb, összefüggő szerkezetű szövegsorozatot generálni.

## Setup

### TensorFlow és más könyvtárak importálása

In [ ]:
import tensorflow as tf

import numpy as np
import os
import time

### Az adatkészlet letöltése

Változtassa meg a következő sort, hogy futtassa ezt a kódot a saját data.c fájlján.

In [ ]:
#path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

### Read the data

Először nézd meg a szöveget:

In [ ]:
# Read, then decode for py2 compat.
text = open('piszkosfred.txt', 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 324730 characters


In [ ]:
# Take a look at the first 250 characters in text
print(text[:250])


Rejtő Jenő

Piszkos Fred, a kapitány

ELSŐ FEJEZET
1
- Uram! A késemért jöttem!
- Hol hagyta?
- Valami matrózban.
- Milyen kés volt?
- Acél. Keskeny penge, kissé hajlott. Nem látta?
- Várjunk... Csak lassan, kérem... Milyen volt a nyele?


In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

96 unique characters


## A szöveg feldolgozása

### A szöveg vektorizálása

A képzés előtt a karakterláncokat numerikus ábrázolásra kell konvertálni. 

Az `tf.keras.layers.StringLookup` réteg képes minden karaktert numerikus azonosítóvá alakítani. Csak előbb tokenekre kell bontani a szöveget.

In [ ]:
example_texts = ['alma', 'szilva']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'l', b'm', b'a'], [b's', b'z', b'i', b'l', b'v', b'a']]>

Most hozzuk létre az `tf.keras.layers.StringLookup` réteget:

In [ ]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

A tokenekről karakterazonosítókra konvertál:

In [ ]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[50, 61, 62, 50], [68, 75, 58, 61, 71, 50]]>

Mivel ennek a bemutatónak az a célja, hogy szöveget generáljon, fontos lesz ezt a reprezentációt megfordítani, és ember által olvasható karakterláncokat visszanyerni belőle. Ehhez használhatjuk az `tf.keras.layers.StringLookup(...., invert=True)` parancsot.  

Megjegyzés: Itt a `sorted(set(text))` segítségével generált eredeti szókincs átadása helyett használjuk az `tf.keras.layers.StringLookup` réteg `get_vocabulary()` módszerét, hogy a `[UNK]` tokenek ugyanúgy legyenek beállítva.

In [ ]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

Ez a réteg visszanyeri a karaktereket az azonosítók vektoraiból, és egy `tf.RaggedTensor` karakterek formájában adja vissza őket:

In [ ]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'l', b'm', b'a'], [b's', b'z', b'i', b'l', b'v', b'a']]>

Az `tf.strings.reduce_join` segítségével a karaktereket visszaillesztheted a karakterláncokba. 

In [ ]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'alma', b'szilva'], dtype=object)

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### A predikciós feladat

Adott egy karakter vagy egy karaktersorozat, mi a legvalószínűbb következő karakter? Ez az a feladat, amire a modellt betanítjuk. A modell bemenete egy karaktersorozat lesz, és a modellt arra képezzük, hogy minden egyes időlépésnél megjósolja a kimenetet - a következő karaktert.

Mivel az RNN-ek fenntartanak egy belső állapotot, amely a korábban látott elemektől függ, az adott pillanatig kiszámított összes karaktert figyelembe véve, mi a következő karakter?


### Képzési példák és célok létrehozása

Ezután ossza a szöveget példasorozatokra. Minden egyes bemeneti szekvencia a szövegből származó `seq_length` karaktereket tartalmazza.

Minden egyes bemeneti szekvenciához a megfelelő célok ugyanolyan hosszúságú szöveget tartalmaznak, kivéve egy karakterrel jobbra eltolva.

Tehát a szöveget `seq_length+1` hosszúságú darabokra bontjuk. Tegyük fel például, hogy a `seq_length` 4, és a szövegünk a "Hello". A bemeneti szekvencia a "Hell", a célszekvencia pedig az "ello" lenne.

Ehhez először használjuk az `tf.data.Dataset.from_tensor_slices` függvényt, hogy a szövegvektort karakterindexek folyamává alakítsuk.

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(324730,), dtype=int64, numpy=array([ 2,  1, 41, ...,  1,  2,  1])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
ids_dataset.take(10)

<TakeDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>

In [ ]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))




R
e
j
t
ő
 
J
e


In [ ]:
seq_length = 100


A "batch" módszerrel ezeket az egyedi karaktereket könnyen átalakíthatja a kívánt méretű szekvenciákká.

In [ ]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'\r' b'\n' b'R' b'e' b'j' b't' b'\xc5\x91' b' ' b'J' b'e' b'n'
 b'\xc5\x91' b'\r' b'\n' b'\r' b'\n' b'P' b'i' b's' b'z' b'k' b'o' b's'
 b' ' b'F' b'r' b'e' b'd' b',' b' ' b'a' b' ' b'k' b'a' b'p' b'i' b't'
 b'\xc3\xa1' b'n' b'y' b'\r' b'\n' b'\r' b'\n' b'E' b'L' b'S' b'\xc5\x90'
 b' ' b'F' b'E' b'J' b'E' b'Z' b'E' b'T' b'\r' b'\n' b'1' b'\r' b'\n' b'-'
 b' ' b'U' b'r' b'a' b'm' b'!' b' ' b'A' b' ' b'k' b'\xc3\xa9' b's' b'e'
 b'm' b'\xc3\xa9' b'r' b't' b' ' b'j' b'\xc3\xb6' b't' b't' b'e' b'm' b'!'
 b'\r' b'\n' b'-' b' ' b'H' b'o' b'l' b' ' b'h' b'a' b'g' b'y' b't' b'a'], shape=(101,), dtype=string)


Könnyebb látni, hogy mit csinál ez, ha a tokeneket visszacsatoljuk karakterláncokká:

In [ ]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'\r\nRejt\xc5\x91 Jen\xc5\x91\r\n\r\nPiszkos Fred, a kapit\xc3\xa1ny\r\n\r\nELS\xc5\x90 FEJEZET\r\n1\r\n- Uram! A k\xc3\xa9sem\xc3\xa9rt j\xc3\xb6ttem!\r\n- Hol hagyta'
b'?\r\n- Valami matr\xc3\xb3zban.\r\n- Milyen k\xc3\xa9s volt?\r\n- Ac\xc3\xa9l. Keskeny penge, kiss\xc3\xa9 hajlott. Nem l\xc3\xa1tta?\r\n- V\xc3\xa1rju'
b'nk... Csak lassan, k\xc3\xa9rem... Milyen volt a nyele?\r\n- Kagyl\xc3\xb3.\r\n- H\xc3\xa1ny r\xc3\xa9szb\xc5\x91l?\r\n- Egy darabb\xc3\xb3l k\xc3\xa9sz\xc3\xbclt.'
b'\r\n- Akkor nincs baj. Megvan a k\xc3\xa9s!\r\n- Hol?\r\n- A h\xc3\xa1tamban.\r\n- K\xc3\xb6sz\xc3\xb6n\xc3\xb6m...\r\n- K\xc3\xa9rem... A csapos mes\xc3\xa9lte'
b', hogy milyen sz\xc3\xa9p k\xc3\xa9s van bennem. Egy darab h\xc3\xbaszcentis kagyl\xc3\xb3ritkas\xc3\xa1g.\r\n- Forduljon meg, k\xc3\xa9rem, hogy'


A képzéshez szükséged lesz egy `(bemenet, címke)` párokból álló adathalmazra. Ahol a `bemenet` és 
`label` szekvenciák. Minden egyes időlépésnél a bemenet az aktuális karakter, a címke pedig a következő karakter. 

Íme egy függvény, amely bemenetként egy szekvenciát vesz, duplikálja és eltolja azt, hogy minden egyes időlépésnél összehangolja a bemenetet és a címkét:

In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'\r\nRejt\xc5\x91 Jen\xc5\x91\r\n\r\nPiszkos Fred, a kapit\xc3\xa1ny\r\n\r\nELS\xc5\x90 FEJEZET\r\n1\r\n- Uram! A k\xc3\xa9sem\xc3\xa9rt j\xc3\xb6ttem!\r\n- Hol hagyt'
Target: b'\nRejt\xc5\x91 Jen\xc5\x91\r\n\r\nPiszkos Fred, a kapit\xc3\xa1ny\r\n\r\nELS\xc5\x90 FEJEZET\r\n1\r\n- Uram! A k\xc3\xa9sem\xc3\xa9rt j\xc3\xb6ttem!\r\n- Hol hagyta'


### Képzési tételek létrehozása

Az `tf.data` segítségével a szöveget kezelhető szekvenciákra osztotta. Mielőtt azonban ezeket az adatokat betáplálnád a modellbe, meg kell keverned az adatokat, és kötegekbe kell csomagolnod őket.

In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## A modell megépítése

Ez a szakasz a modellt a `keras.Model` alosztályként definiálja (A részletekért lásd [Új rétegek és modellek létrehozása alosztályozással](https://www.tensorflow.org/guide/keras/custom_layers_and_models)). 

Ez a modell három réteggel rendelkezik:

* `tf.keras.layers.Embedding`: A bemeneti réteg. Egy betanítható keresőtábla, amely minden karakterazonosítót egy `embedding_dim` dimenziójú vektorra képez le;
* `tf.keras.layers.GRU`: Egyfajta RNN, amelynek mérete `units=rnn_units` (Itt egy LSTM réteget is használhatsz.)
* `tf.keras.layers.Dense`: A kimeneti réteg, `vocab_size` kimenetekkel. A szókészlet minden egyes karakterére egy logaritást ad ki. Ezek az egyes karakterek log-valószínűségét adják meg a modell szerint.

In [ ]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
    return model

#Ez a kaggleről van

In [ ]:
'''model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)'''

model = build_model(vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units, batch_size=BATCH_SIZE)

A modell minden egyes karakterhez megnézi a beágyazást, lefuttatja a GRU-t egy időlépést a beágyazással, és a sűrű réteget alkalmazza a következő karakter logaritmusát előrejelző logaritmusok létrehozására:

![A drawing of the data passing through the model](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_training.png?raw=1)

Megjegyzés: A képzéshez használhat egy `keras.Sequential` modellt. A későbbi szöveggeneráláshoz az RNN belső állapotát kell majd kezelned. Egyszerűbb előre felvenni az állapot bemeneti és kimeneti beállításait, mint később átrendezni a modell architektúráját. További részletekért lásd a [Keras RNN útmutató](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse).

## Próbálja ki a modellt

Most futtassa a modellt, hogy lássa, a várakozásoknak megfelelően viselkedik-e.

Először ellenőrizze a kimenet alakját:

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 97) # (batch_size, sequence_length, vocab_size)


A fenti példában a bemenet szekvencia hossza "100", de a modell bármilyen hosszúságú bemenettel futtatható:

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (64, None, 256)           24832     
                                                                 
 gru_1 (GRU)                 (64, None, 1024)          3938304   
                                                                 
 dense_1 (Dense)             (64, None, 97)            99425     
                                                                 
Total params: 4,062,561
Trainable params: 4,062,561
Non-trainable params: 0
_________________________________________________________________


Ahhoz, hogy tényleges előrejelzéseket kapjunk a modellből, mintát kell vennünk a kimeneti eloszlásból, hogy megkapjuk a tényleges karakterindexeket. Ezt az eloszlást a karakterszókincs logaritmusai határozzák meg.

Megjegyzés: Fontos, hogy ebből az eloszlásból _mintavételezzünk_, mivel az eloszlás _argmax_ értékét véve a modell könnyen hurokba kerülhet.

Próbáljuk ki a tétel első példájánál:

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

Ez minden egyes időlépésnél megadja a következő karakterindex előrejelzését:

In [ ]:
sampled_indices

array([14,  2,  2, 80, 76, 57, 77, 78, 39, 69, 29, 95, 27, 71, 83, 81, 81,
       82, 34, 26, 33, 10, 62, 54, 62, 58, 71, 31, 46, 31, 20, 65,  7, 78,
       63, 15, 78, 67, 41, 72, 54, 56, 54, 28, 93, 19, 76, 78, 34, 29, 65,
       84, 74, 87,  6, 27, 41, 76, 17, 67, 62, 45, 73, 22, 83, 30, 78, 88,
       88, 68, 69, 58, 19, 33,  1, 50, 64, 81, 14, 30,  2, 26, 96, 60, 72,
       66, 64,  5, 32, 50, 96, 55, 89, 18, 45, 81, 32, 91, 41, 89])

Dekódolja ezeket, hogy lássa a nem képzett modell által megjósolt szöveget:

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'.\r\nMr. Irving mindent feljegyzett egy piros noteszbe, \xc3\xa9s elhat\xc3\xa1rozta, hogy meg is tanulja.\r\n- A Vel\xc5\x91'

Next Char Predictions:
 b'2\r\r\xc3\x96\xc3\x81h\xc3\x89\xc3\x8dOtE\xe2\x80\x9cCv\xc3\xa1\xc3\x9a\xc3\x9a\xc3\x9cJBI-memivGWG8p)\xc3\x8dn3\xc3\x8drRwegeD\xc5\xb17\xc3\x81\xc3\x8dJEp\xc3\xa9y\xc3\xb6(CR\xc3\x815rmVx:\xc3\xa1F\xc3\x8d\xc3\xba\xc3\xbasti7I\nao\xc3\x9a2F\rB\xe2\x80\x9dkwqo!Ha\xe2\x80\x9df\xc3\xbc6V\xc3\x9aH\xc5\x91R\xc3\xbc'


## A modell betanítása

Ezen a ponton a probléma egy szokásos osztályozási problémaként kezelhető. Az előző RNN-állapot és a bemeneti adatok ismeretében ebben az időlépésben meg kell jósolni a következő karakter osztályát.

### Csatoljunk egy optimalizálót és egy veszteségfüggvényt.

A szabványos `tf.keras.losses.sparse_categorical_crossentropy` veszteségfüggvény ebben az esetben működik, mivel az előrejelzések utolsó dimenziójára alkalmazzák.

Mivel a modelled logitokat ad vissza, be kell állítanod a `from_logits` flaget.


In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 97)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.574665, shape=(), dtype=float32)


Egy újonnan inicializált modellnek nem szabad túlságosan biztosnak lennie önmagában, a kimeneti logaritmusoknak mind hasonló nagyságúnak kell lenniük. Ennek megerősítésére ellenőrizheti, hogy az átlagos veszteség exponenciálisa megközelítőleg megegyezik-e a szókincs méretével. Egy sokkal nagyobb veszteség azt jelenti, hogy a modell biztos a rossz válaszaiban, és rosszul van inicializálva:

In [ ]:
tf.exp(example_batch_mean_loss).numpy()

96.995544

Konfigurálja a képzési eljárást az `tf.keras.Model.compile` módszerrel. Használja az `tf.keras.optimizers.Adam` modellt alapértelmezett argumentumokkal és a veszteségfüggvényt.

In [ ]:
model.compile(optimizer='adam', loss=loss)

### Ellenőrző pontok konfigurálása

A`tf.keras.callbacks.ModelCheckpoint` használatával biztosíthatja, hogy az ellenőrzési pontok a képzés során mentésre kerüljenek:

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### A képzés elvégzése

A képzési idő ésszerűségének megőrzése érdekében használjon 10 epochát a modell képzéséhez. A Colabban a gyorsabb képzés érdekében állítsa a futási időt GPU-ra.

In [ ]:
EPOCHS = 30

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
50/50 [==============================] - 5s 53ms/step - loss: 3.7242
Epoch 2/30
50/50 [==============================] - 3s 52ms/step - loss: 2.6969
Epoch 3/30
50/50 [==============================] - 3s 52ms/step - loss: 2.4494
Epoch 4/30
50/50 [==============================] - 3s 52ms/step - loss: 2.3331
Epoch 5/30
50/50 [==============================] - 3s 51ms/step - loss: 2.2335
Epoch 6/30
50/50 [==============================] - 3s 52ms/step - loss: 2.1369
Epoch 7/30
50/50 [==============================] - 3s 52ms/step - loss: 2.0405
Epoch 8/30
50/50 [==============================] - 3s 52ms/step - loss: 1.9473
Epoch 9/30
50/50 [==============================] - 3s 53ms/step - loss: 1.8551
Epoch 10/30
50/50 [==============================] - 3s 53ms/step - loss: 1.7730
Epoch 11/30
50/50 [==============================] - 3s 53ms/step - loss: 1.6937
Epoch 12/30
50/50 [==============================] - 3s 52ms/step - loss: 1.6196
Epoch 13/30
50/50 [==================

## Szöveg generálása

A legegyszerűbben úgy generálhatsz szöveget ezzel a modellel, ha egy ciklusban futtatod, és a végrehajtás során nyomon követed a modell belső állapotát.

![A szöveg generálásához a modell kimenete visszakerül a bemenetre](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_sampling.png?raw=1)

A modell minden egyes meghívásakor átadunk egy szöveget és egy belső állapotot. A modell a következő karakterre vonatkozó előrejelzést és annak új állapotát adja vissza. A szöveggenerálás folytatásához adja vissza a predikciót és az állapotot.


Az alábbiakban egylépéses előrejelzést teszünk:

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Futtassa le egy ciklusban, hogy létrehozzon egy szöveget. Ha megnézzük a generált szöveget, láthatjuk, hogy a modell tudja, mikor kell nagybetűvel írni, bekezdéseket alkotni, és Rejtő-szerű írásszókincset utánoz. A kevés gyakorló epocha miatt még nem tanulta meg, hogy összefüggő mondatokat alkosson.

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['A'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

An saját magaszására társal lett van a pitánsággal, könnyedéseket.
A herceg szeme le régen is tűt. Mehes. Fülig Jimmy? Mi ajra, ha Hurcunk Fernántesz! És igen ötök boripán az találgassza, Felség, angol hibetkézik az én nevemben, azt hitte, hogy néhány óráj baj vas, hát de nem hites, előfordul egy erőtel sem mozdult. Pedig a hajón vérn A fiú uralkodásai és lenéssel az illetőkeá.
- Hány és GrAndszervez erted?
Így öreg Wilson Hutchins (az amerikai fűtő már csak azért sem vonállattag érezte, amelyen a pincér aztán ki történt. Fel tudum. Most már közzem (Övig nem ívás, hanem Jiment furcsa, akkor ja mokdanás, mert a brót és fél személyesen ismerem.
- Warins a vőlegverék el - mondja a stamát... Az is a pillanatra! - jegyezte meg közölte?
- Ezent lesz a borízsal állt. - Es osztálybal. Öngyen közt kenyeres tudja, ő a próféte megtoválját, így szólt:
- Integessen Felség, hogy egy nap alatt sok mindent tett, embere! Hozzt, hogy nem keresem tovább vezeteti Bannera. Tevissza kelé krabitány az á 

__

In [ ]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

In [ ]:
def generate_text(model, start_string):  
  
    num_generate = 1000
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

   
    text_generated = []

    temperature = 1.0

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u"A"))

ValueError: ignored

A legegyszerűbb dolog, amit tehetsz az eredmények javítása érdekében, hogy hosszabb ideig edzed (próbáld ki az `EPOCHS = 30`).

Kísérletezhet más kezdő stringgel is, megpróbálhat egy másik RNN réteget hozzáadni a modell pontosságának javítása érdekében, vagy beállíthatja a hőmérséklet paramétert, hogy több vagy kevesebb véletlenszerű előrejelzést generáljon.

## Export the generator

This single-step model can easily be [saved and restored](https://www.tensorflow.org/guide/saved_model), allowing you to use it anywhere a `tf.saved_model` is accepted.

In [ ]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')